# Capstone project Coursera

#### Packages used

In [38]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

## Scraping data from a wiki table and create dataframe

In [64]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#requesting page online
html_page = requests.get(link).text

#scraping required data from the webpage
soup = BeautifulSoup(html_page, 'html.parser')
wiki_table = soup.find('table',{'class':'wikitable'})
RawData_postalcode_0 = wiki_table.findAll('tr')

#cleaning the data
RawData_postalcode_1 = [i.text.strip() for i in ddata]
Postal_code_Canada = [i.split('\n\n') for i in data_list]

#Creating of dataframe
df = pd.DataFrame(Postal_code_Canada)
df.columns = df.iloc[0]
df = df[1:]
df=df[df.Borough != 'Not assigned']
df.head()

,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [80]:
df.shape

(103, 3)